<a href="https://www.kaggle.com/code/kentvejrupmadsen/tensorflow-image-classification-of-letters?scriptVersionId=133556420" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [22]:
%pip install ipykernel
!python -m ipykernel install --user --name=notebook_environment

%pip install matplotlib
%pip install keras
%pip install tensorflow
%pip install kaggle
%pip install numpy
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Installed kernelspec notebook_environment in /root/.local/share/jupyter/kernels/notebook_environment
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [24]:
from random \
    import SystemRandom

import tensorflow

from tensorflow.data \
    import AUTOTUNE

from keras.losses \
    import SparseCategoricalCrossentropy

from keras.utils \
    import image_dataset_from_directory

from keras \
    import \
    Model, \
    Sequential

from keras \
    import layers

import wandb

from wandb.integration.keras \
    import \
    WandbCallback, \
    WandbEvalCallback, \
    WandbMetricsLogger, \
    WandbModelCheckpoint

In [ ]:
# Constants
def label_training() -> str:
    return 'training'

def label_validation() -> str:
    return 'validation'


In [32]:
configuration = {
    'batch_size': 18,
    'epochs': 10,
    
    'view': {
        'height': 512,
        'width': 512,
        'channels': 3
    },
    
    'dataset':
    {
        'split_validation_at': 0.25,
        
        'training':
        {
            'labels': [],
            'size': 0
        },
        
        'validation':
        {
            'labels': [],
            'size': 0
        }
    },
    
    'seed': generate_random_seed()
}

In [ ]:
def set_size_of_training_set_categories(
    value: int
) -> None:
    global configuration
    configuration['dataset']['training']['size'] = value

def set_labels_of_training_set_categories(
    value: list
) -> None:
    global configuration
    configuration['dataset']['training']['labels'] = value
    set_size_of_training_set_categories(
        len(value)
    )


In [ ]:
def set_size_of_validation_set_categories(
    value: int
) -> None:
    global configuration
    configuration['dataset']['validation']['size'] = value

def set_labels_of_validation_set_categories(
    value: list
) -> None:
    global configuration
    configuration['dataset']['validation']['labels'] = value
    
    set_size_of_validation_set_categories(
        len(value)
    )


In [28]:
#
use_cache: bool = False
log_runs: bool = False

wandb_callback = None

r_generator = SystemRandom()

cache_buffer_size: int = AUTOTUNE

vision_height: int = configuration['view']['height']
vision_width: int = configuration['view']['width']

vision_number_of_color_channels: int = configuration['view']['channels']

vision_size = (vision_height, vision_width) 
vision_size_w_channels = (vision_height, vision_width, vision_number_of_color_channels)

split_dataset_at: float = configuration['dataset']['split_validation_at']

dataset_seed: int = configuration['seed']
dataset_batch_size: int = configuration['batch_size']

path_to_dataset: str = '../input/letter-images-dataset/dataset'

training_dataset = None
validation_dataset = None

pointer_to_training_dataset = None
pointer_to_validation_dataset = None

model = None

# Callbacks

# result
results: list = []

In [ ]:
def get_log_runs() -> bool:
    global log_runs
    return log_runs

def generate_random_seed() -> int:
    global r_generator
    return r_generator.randint(1, 750000)

In [27]:
strategy = tensorflow.distribute.MirroredStrategy(
    devices=[
        '/gpu:0', 
        '/gpu:1'
    ]
)

In [29]:
if get_log_runs():
    wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [36]:
training_dataset = image_dataset_from_directory(
    path_to_dataset,
    validation_split = split_dataset_at,
    subset= label_training(),
    seed = dataset_seed,
    image_size = vision_size,
    batch_size = dataset_batch_size,
    shuffle=True,
    crop_to_aspect_ratio=True
)

validation_dataset = image_dataset_from_directory(
    path_to_dataset,
    validation_split = split_dataset_at,
    subset= label_validation(),
    seed = dataset_seed,
    image_size = vision_size,
    batch_size = dataset_batch_size,
    shuffle=True,
    crop_to_aspect_ratio=True
)


Found 11339 files belonging to 67 classes.
Using 8505 files for training.
Found 11339 files belonging to 67 classes.
Using 2834 files for validation.


In [38]:
if get_log_runs():
    wandb.init(
        project='letter-identification',
        entity='designermadsen', 
        config=configuration,
        sync_tensorboard=True,
        save_code=True,
        monitor_gym=True
    )

In [39]:
category_size = configuration['dataset']['training']['size'] 

In [40]:
def make_model():
    global \
        category_size, \
        vision_size_w_channels
    
    division = (2, 2)
    
    return Sequential(
            [
                layers.Rescaling(
                    1./255, 
                    input_shape=vision_size_w_channels, 
                    trainable=False
                ),
                
                layers.Conv2D(
                    256, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),
                
                layers.Conv2D(
                    128, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3,
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64,
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),
                
                layers.Flatten(),
                layers.Dense(
                    (category_size * 8), 
                    activation='relu',
                ),
                
                layers.Dense(
                    category_size
                )
            ]
        )

In [41]:
if not (strategy is None):
    with strategy.scope():    
        model = make_model()
else:
    model = make_model()

In [42]:
model.compile(
    optimizer='adam', 
    loss=SparseCategoricalCrossentropy(
        from_logits=True
    ), 
    metrics=['accuracy'], 
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 512, 512, 3)       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 512, 512, 256)     7168      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 256, 256, 256)    0         
 2D)                                                             
                                                                 
 conv2d_23 (Conv2D)          (None, 256, 256, 128)     295040    
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 128, 128, 128)    0         
 2D)                                                             
                                                                 
 conv2d_24 (Conv2D)          (None, 128, 128, 64)     

In [43]:
if use_cache:
    pointer_to_training_dataset = training_dataset.cache().prefetch(
        buffer_size=cache_buffer_size
    )
    
    pointer_to_validation_dataset = validation_dataset.cache().prefetch(
        buffer_size=cache_buffer_size
    )
else:
    pointer_to_training_dataset = training_dataset
    pointer_to_validation_dataset = validation_dataset

In [44]:
if get_log_runs():
    wandb_callback = WandbCallback(
        log_weights=True, 
        log_gradients=True, 
        training_data=pointer_to_training_dataset, 
        validation_data=pointer_to_validation_dataset,

        log_evaluation=True, 

        monitor='val_accuracy',
        mode='max'
    )

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [45]:
epochs = configuration['epochs']
fit_callbacks = []

if not (wandb_callback is None):
    fit_callbacks.append(wandb_callback)

In [46]:
history = model.fit(
    pointer_to_training_dataset,
    validation_data = pointer_to_validation_dataset,
    epochs=epochs,
    callbacks=fit_callbacks
)

wandb: WARNING Error initializing ValidationDataLogger in WandbCallback. Skipping logging validation data. Error: '_BatchDataset' object is not subscriptable


Epoch 1/10
473/473 [==============================] - ETA: 0s - loss: 3.8881 - accuracy: 0.0561

wandb: Adding directory to artifact (/kaggle/working/wandb/run-20230614_100353-rcl58nc7/files/model-best)... Done. 0.1s


473/473 [==============================] - 602s 1s/step - loss: 3.8881 - accuracy: 0.0561 - val_loss: 3.8638 - val_accuracy: 0.0642
Epoch 2/10
473/473 [==============================] - 542s 1s/step - loss: 3.8585 - accuracy: 0.0609 - val_loss: 3.8605 - val_accuracy: 0.0642
Epoch 3/10
473/473 [==============================] - 545s 1s/step - loss: 3.8555 - accuracy: 0.0609 - val_loss: 3.8552 - val_accuracy: 0.0642
Epoch 4/10
473/473 [==============================] - 550s 1s/step - loss: 3.8540 - accuracy: 0.0609 - val_loss: 3.8564 - val_accuracy: 0.0642
Epoch 5/10
473/473 [==============================] - 591s 1s/step - loss: 3.8534 - accuracy: 0.0609 - val_loss: 3.8549 - val_accuracy: 0.0642
Epoch 6/10
473/473 [==============================] - 542s 1s/step - loss: 3.8516 - accuracy: 0.0600 - val_loss: 3.8542 - val_accuracy: 0.0642
Epoch 7/10
473/473 [==============================] - 586s 1s/step - loss: 3.8514 - accuracy: 0.0609 - val_loss: 3.8534 - val_accuracy: 0.0642
Epoch 8/10

In [49]:
path_to_model: str = '/kaggle/working'

In [51]:
result = {
    'validation': {
        'accuracy': history.history['val_accuracy'],
        'loss': history.history['val_loss']
    },
    'result': {
        'accuracy': history.history['accuracy'],
        'loss': history.history['loss']
    }
}

results.append(result)

In [53]:
wandb.log({
    'training': results
})

In [57]:
model.save(
    path_to_model
)

In [58]:
wandb.finish()

accuracy,▁████▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▃▄▃▂▂▂▂▁
accuracy,0.06091
best_epoch,0
best_val_accuracy,0.06422
epoch,9
loss,3.84987
val_accuracy,0.06422
